# Compute Metrics
Precomputes every (model viz, benchmark viz) pair and performs an analysis.
For visualization similarity, uses the structural similarity index measure (SSIM). 

In [1]:
# Go up a directory so we have access to the imports and files in the parent
# directory
import dask
from dask.distributed import Client, LocalCluster
from server.model_setup import get_ncNetInstance, get_nl4dv_instance
import multiprocessing
import logging
import warnings
import json
import os
warnings.filterwarnings("ignore")

BENCHMARK_DIR_PATH = os.path.join("server", "assets", "benchmark")
BENCHMARK_DATA_DIR_PATH = os.path.join(BENCHMARK_DIR_PATH, "data")


c:\Users\casil\Documents\Spring_2022\6.S079\final-project\server\model_setup.py


In [9]:
from vega import VegaLite
import altair as alt
import panel as pn
import json

pn.extension("vega")

bench_spec = {
    "data": {
        "values": [
            {
                "aircraft_id": 1,
                "fleet_series": "444-464 (21)",
                "fuel_propulsion": "Diesel",
                "manufacturer": "Gillig",
                "model": "Phantom (High Floor)",
                "order_year": 1992,
                "powertrain": "DD S50EGR Allison WB-400R",
            },
            {
                "aircraft_id": 2,
                "fleet_series": "465-467 (3)",
                "fuel_propulsion": "Diesel",
                "manufacturer": "Gillig",
                "model": "Phantom (High Floor)",
                "order_year": 1996,
                "powertrain": "DD S50 Allison WB-400R",
            },
            {
                "aircraft_id": 3,
                "fleet_series": "468-473 (6)",
                "fuel_propulsion": "Diesel",
                "manufacturer": "Gillig",
                "model": "Phantom (High Floor)",
                "order_year": 1998,
                "powertrain": "DD S50 Allison WB-400R",
            },
            {
                "aircraft_id": 4,
                "fleet_series": "474-481 (8)",
                "fuel_propulsion": "Diesel",
                "manufacturer": "Gillig",
                "model": "Advantage (Low Floor)",
                "order_year": 2000,
                "powertrain": "Cummins ISC Allison WB-400R",
            },
            {
                "aircraft_id": 5,
                "fleet_series": "482-492 (11)",
                "fuel_propulsion": "Diesel",
                "manufacturer": "Gillig",
                "model": "Advantage (Low Floor)",
                "order_year": 2002,
                "powertrain": "Cummins ISL Allison WB-400R",
            },
            {
                "aircraft_id": 6,
                "fleet_series": "300-309 (10)",
                "fuel_propulsion": "Hybrid",
                "manufacturer": "NFI",
                "model": "GE40LFR",
                "order_year": 2010,
                "powertrain": "Ford Triton V10 ISE-Thundervolt TB40-HG",
            },
            {
                "aircraft_id": 7,
                "fleet_series": "310-329 (20)",
                "fuel_propulsion": "CNG",
                "manufacturer": "NFI",
                "model": "C40LFR",
                "order_year": 2011,
                "powertrain": "Cummins Westport ISL-G Allison WB-400R",
            },
        ]
    },
    "encoding": {
        "color": {"field": "model", "type": "nominal"},
        "theta": {
            "aggregate": "count",
            "field": "description",
            "type": "quantitative",
        },
    },
    "mark": "arc",
}

spec = {
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "mark": {"type": "line", "tooltip": True},
    "encoding": {
        "x": {
            "field": "DepartmentID",
            "type": "quantitative",
            "aggregate": None,
            "bin": True,
            "axis": {"format": "s"},
        },
        "y": {
            "field": "DepartmentID",
            "type": "quantitative",
            "aggregate": "count",
            "axis": {"format": "s"},
        },
    },
    "transform": [],
    "data": {
        "url": "server\\assets\\benchmark\\data\\department.csv",
        "format": {"type": "csv"},
    },
}

viz = VegaLite(spec)

# print(alt.VegaLite(spec).save("test.html"))
# Save the panel to a png
# pn.panel(spec, width=600, height=400).save("test.json")
with open("test.json", "w") as f:
    json.dump(spec, f)

import subprocess

# subprocess.run("npm install vega-lite", shell=True)
subprocess.run(
    "npx -p vega -p vega-lite vl2png test.json test.png".split(),
    shell=True,
    capture_output=True,
)
# print(viz.render_type)


CompletedProcess(args=['npx', '-p', 'vega', '-p', 'vega-lite', 'vl2png', 'test.json', 'test.png'], returncode=0, stdout=b'', stderr=b'')

In [12]:
import uuid


def get_viz_metrics(spec1, spec2, cleanup=True):
    spec1["autosize"] = "fit"
    spec1["width"] = 500
    spec1["height"] = 500
    spec1["background"] = "#fafafa"

    spec2["autosize"] = "fit"
    spec2["width"] = 500
    spec2["height"] = 500
    spec2["background"] = "#fafafa"

    pair_id = str(uuid.uuid4())
    # Model result spec
    spec1_json_path = f"spec1-{pair_id}.json"
    with open(spec1_json_path, "w") as f:
        json.dump(spec1, f)

    # Benchmark spec
    spec2_json_path = f"spec2-{pair_id}.json"
    with open(spec2_json_path, "w") as f:
        json.dump(spec2, f)

    # Convert the specs to png using the vega-lite CLI
    spec1_png_path = f"spec1-{pair_id}.png"
    subprocess.run(
        f"npx -p vega -p vega-lite vl2png {spec1_json_path} {spec1_png_path}".split(),
        shell=True,
    )
    spec2_png_path = f"spec2-{pair_id}.png"
    subprocess.run(
        f"npx -p vega -p vega-lite vl2png {spec2_json_path} {spec2_png_path}".split(),
        shell=True,
    )
    
    if not cleanup:
        return
    
    os.remove(spec1_json_path)
    os.remove(spec2_json_path)
    # os.remove(spec1_png_path)
    # os.remove(spec2_png_path)
        

In [13]:
get_viz_metrics(spec, bench_spec)

Here we create a JSON lookup mapping datasets to the NL queries from the benchmark.

In [5]:
benchmark_meta_path = os.path.join(BENCHMARK_DIR_PATH, "benchmark_meta.json")
table_to_benchmark_lookup_path = os.path.join(
    BENCHMARK_DIR_PATH, "table_to_benchmark_lookup.json"
)

with open(benchmark_meta_path, "r") as file:
    benchmark_metadata: dict = json.load(file)

with open(table_to_benchmark_lookup_path, "r") as file:
    lookup = json.load(file)

dataset_to_queries_lookup = {}
for dataset_name, benchmark_ids in lookup.items():
    # Get all of the NL queries for the dataset
    nl_queries = [
        nl_query
        for benchmark_id in benchmark_ids
        for nl_query in benchmark_metadata[benchmark_id]["nl_queries"]
    ]
    
    dataset_to_queries_lookup[dataset_name] = nl_queries
    
# Save the dataset_to_queries_lookup to a file
with open(os.path.join(BENCHMARK_DIR_PATH, "dataset_to_queries_lookup.json"), "w") as file:
    json.dump(dataset_to_queries_lookup, file, indent=4)
    

Create the Dask client for multiprocessing

In [2]:
# Get the number of cores
n_cores = multiprocessing.cpu_count()
print("Number of cores we have: ", n_cores)

# Create a cluster and client
print("> Creating a cluster and client...")
cluster = LocalCluster(
    ip=None,
    n_workers=n_cores,
    processes=True,
    silence_logs=logging.ERROR,
    # interface="lo",
)
client = Client(cluster)
client

Number of cores we have:  12
> Creating a cluster and client...


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:55843/status,
Dashboard: http://127.0.0.1:55843/status,Workers: 12
Total threads: 12,Total memory: 15.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55844,Workers: 12
Dashboard: http://127.0.0.1:55843/status,Total threads: 12
Started: Just now,Total memory: 15.79 GiB
Comm: tcp://127.0.0.1:55958,Total threads: 1
Dashboard: http://127.0.0.1:55959/status,Memory: 1.32 GiB
Nanny: tcp://127.0.0.1:55857,


In [12]:
def compute_metrics(model_name, dataset_name):
    """The primary computation function to be parallelized."""
    dataset = dataset_name.replace(".csv", "") + ".csv"
    data_path = os.path.join(BENCHMARK_DATA_DIR_PATH, dataset)

    nl_queries = dataset_to_queries_lookup[dataset_name]

    try:
        if model_name == "nl4dv":
            model = get_nl4dv_instance(data_path=data_path)
        elif model_name == "ncNet":
            model = get_ncNetInstance(data_path=data_path)
    except FileNotFoundError:
        print('File not found: ', data_path)
        return None

    result = {'model_name': model_name, 'dataset_name': dataset_name, 'results': []}
    for i, nl_query in enumerate(nl_queries):
        # Temporary stopping measure
        vl_spec = None
        if model_name == "nl4dv":
            print("EXECUTING QUERY:", nl_query, "on nl4dv")
            model_result = model.analyze_query(nl_query)
            # Get the first VegaLite spec
            vis_list = model_result["visList"]
            if len(vis_list) > 0:
                vl_spec = vis_list[0]["vlSpec"]
            else:
                vl_spec = None # TODO: change this to a boolean
                
        elif model_name == "ncNet":
            viz = model.nl2vis(nl_query)[
                0
            ]  # nl2vis will return a list a [Vis, VegaLiteSpec]

            vl_spec = viz.spec
            
        result['results'].append({
            'nl_query': nl_query,
            'vl_spec': vl_spec,
            'ssim': 0,
        })
    return result

In [13]:
parameters = [
    (model_name, dataset_name)
    for model_name in ["nl4dv", "ncNet"]
    for dataset_name in dataset_to_queries_lookup.keys()
]

lazy_results = []
for i, (model_name, dataset_name) in enumerate(parameters):
    # Temporary stopping measure
    if i >= 2:
        break
    lazy_result = dask.delayed(compute_metrics)(model_name, dataset_name)
    lazy_results.append(lazy_result)
    print(lazy_result)

for result in dask.compute(*lazy_results):
    print(result)

Delayed('compute_metrics-748ea98c-d0e1-4083-8a6a-b3b12bc1c830')
Delayed('compute_metrics-05a22352-063e-41ac-acee-f2681b645e4b')
None
{'model_name': 'nl4dv', 'dataset_name': 'department', 'results': [{'nl_query': 'Show me a line chart of how the number of creation change over the creation, sort by the Creation in descending.', 'vl_spec': None, 'ssim': 0}, {'nl_query': 'Show the number of departments created in different year and bin creation time by year in a line chart, list x axis in descending order.', 'vl_spec': {'$schema': 'https://vega.github.io/schema/vega-lite/v4.json', 'mark': {'type': 'line', 'tooltip': True}, 'encoding': {'x': {'field': 'DepartmentID', 'type': 'quantitative', 'aggregate': None, 'bin': True, 'axis': {'format': 's'}}, 'y': {'field': 'DepartmentID', 'type': 'quantitative', 'aggregate': 'count', 'axis': {'format': 's'}}}, 'transform': [], 'data': {'url': 'server\\assets\\benchmark\\data\\department.csv', 'format': {'type': 'csv'}}}, 'ssim': 0}, {'nl_query': 'Show